In [1]:
import math
import tensorflow as tf
import numpy as np
import matplotlib
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)
xy_with_noise = np.genfromtxt('/Users/yeseo/Desktop/2015eliminated_1.txt',delimiter = ',',dtype = None)

#data_preprocessing
xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
b= MinMaxScaler(b)
xy = np.hstack((a,b))

xy_with_noise = xy_with_noise[:,:27]
a_with_noise = xy_with_noise[:,:2]
b_with_noise = xy_with_noise[:,2:]
b_with_noise = MinMaxScaler(b_with_noise)
xy_with_noise = np.hstack((a_with_noise,b_with_noise))


#parameters
seq_length =6
data_dim =27
hidden_dim = 25
output_dim = 25
learning_rate = 0.01
iterations = 4000

train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

#divide data set to train,validation and test set
train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

train_set_with_noise = xy_with_noise[:train_size]
validation_set_with_noise = xy_with_noise[train_size:train_size+validation_size]
test_set_with_noise = xy_with_noise[train_size+validation_size:]

# build data set for rnn
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

trainX_with_noise, trainY_with_noise = build_dataset(train_set_with_noise,seq_length)
validationX_with_noise, validationY_with_noise = build_dataset(validation_set_with_noise,seq_length)
testX_with_noise,testY_with_noise = build_dataset(test_set_with_noise, seq_length)


X1 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y1 = tf.placeholder(tf.float32,[None,25])

X2 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y2 = tf.placeholder(tf.float32,[None,25])

#LSTM CELL만들기

with tf.variable_scope("rnn1"):
    cell1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs1,_states1 = tf.nn.dynamic_rnn(cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[:,-1], output_dim,activation_fn = None)

    loss1 =tf.reduce_sum(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)

with tf.variable_scope("rnn2"):
    cell2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs2,_states2 = tf.nn.dynamic_rnn(cell2, X2, dtype = tf.float32)
    Y_pred_with_noise = tf.contrib.layers.fully_connected(outputs2[:,-1], output_dim,activation_fn = None)

    loss2 =tf.reduce_mean(tf.square(Y_pred_with_noise-Y2))
    train2 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss2)


#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
x2 = x1+0.3
x3 = x2+0.3
loss_for_graph = np.zeros(iterations)
x4 = np.array(range(0,iterations))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY})
        print("[step: {}] loss: {}".format(i,step_loss1))
        loss_for_graph[i] = step_loss1
        _, step_loss2 = sess.run([train2,loss2],feed_dict={X2:trainX_with_noise, Y2:trainY_with_noise})
        print("[step: {}] loss: {}".format(i,step_loss2))
        
    test_predict = sess.run(Y_pred, feed_dict = {X1:validationX})
    test_predict_with_noise = sess.run(Y_pred_with_noise, feed_dict = {X2:validationX_with_noise})

    
    rmse_val = sess.run(rmse, feed_dict={targets: validationY,predictions: test_predict})
    print("RMSE: {}".format(rmse_val))
   # print("pred: {}".format(test_predict[-1,:]))
    #print("real: {}".format(testY[-1,:]))
    #print("noise: {}".format(eliminate_noise_pred[-1,:]))
    
#    plt.bar(x1,test_predict[-1,:],label = 'predict',color ='b',width = 0.1)
  #  plt.bar(x2,testY[-1,:],label = 'real',color ='g',width = 0.1)
    #plt.bar(x3,eliminate_noise_pred[-1,:],label = 'noise',color ='g',width = 0.1)
    plt.plot(x4,loss_for_graph)
    #plt.legend()
    plt.show()



Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
[step: 0] loss: 19893.150390625
[step: 0] loss: 0.3186686038970947
[step: 1] loss: 13350.0615234375
[step: 1] loss: 0.22086718678474426
[step: 2] loss: 9445.087890625
[step: 2] loss: 0.1540890485048294
[step: 3] loss: 6837.685546875
[step: 3] loss: 0.11645430326461792
[step: 4] loss: 5063.9072265625
[step: 4] loss: 0.09055017679929733
[step: 5] loss: 3818.775634765625
[step: 5] loss: 0.07259102165699005
[step: 6] loss: 3022.6396484375
[step: 6] loss: 0.05909343436360359
[step: 7] loss: 2605.7705078125
[step: 7] loss: 0.04891415312886238
[step: 8] loss: 2438.03076171875
[step: 8] loss: 0.041822079569101334
[step: 9] loss: 2329.1484375
[step: 9] loss: 0.03690825775265694
[step: 10] loss: 2277.142333984375
[step: 10] loss: 0.03307557478547096
[step: 11] loss: 2

[step: 109] loss: 187.52374267578125
[step: 109] loss: 0.007729025557637215
[step: 110] loss: 185.93887329101562
[step: 110] loss: 0.007723530754446983
[step: 111] loss: 187.21856689453125
[step: 111] loss: 0.00771965179592371
[step: 112] loss: 188.7935791015625
[step: 112] loss: 0.00771602476015687
[step: 113] loss: 187.3172149658203
[step: 113] loss: 0.007710547186434269
[step: 114] loss: 185.3411102294922
[step: 114] loss: 0.007701356895267963
[step: 115] loss: 182.25384521484375
[step: 115] loss: 0.0076873572543263435
[step: 116] loss: 180.2296142578125
[step: 116] loss: 0.007671221625059843
[step: 117] loss: 180.17929077148438
[step: 117] loss: 0.007656156551092863
[step: 118] loss: 180.7012176513672
[step: 118] loss: 0.0076451716013252735
[step: 119] loss: 181.52059936523438
[step: 119] loss: 0.007638854905962944
[step: 120] loss: 181.39195251464844
[step: 120] loss: 0.00763559527695179
[step: 121] loss: 181.34835815429688
[step: 121] loss: 0.007633374072611332
[step: 122] loss: 

[step: 218] loss: 0.007290774490684271
[step: 219] loss: 158.1293487548828
[step: 219] loss: 0.007288978900760412
[step: 220] loss: 137.6046142578125
[step: 220] loss: 0.007286765147000551
[step: 221] loss: 149.46331787109375
[step: 221] loss: 0.007284283172339201
[step: 222] loss: 152.0477752685547
[step: 222] loss: 0.007281993515789509
[step: 223] loss: 144.03697204589844
[step: 223] loss: 0.007280061021447182
[step: 224] loss: 138.84719848632812
[step: 224] loss: 0.007278169970959425
[step: 225] loss: 149.42950439453125
[step: 225] loss: 0.007276056334376335
[step: 226] loss: 148.7539520263672
[step: 226] loss: 0.007273765746504068
[step: 227] loss: 139.73953247070312
[step: 227] loss: 0.007271537557244301
[step: 228] loss: 136.92869567871094
[step: 228] loss: 0.00726949330419302
[step: 229] loss: 145.20587158203125
[step: 229] loss: 0.007267565466463566
[step: 230] loss: 141.76705932617188
[step: 230] loss: 0.007265596184879541
[step: 231] loss: 133.40744018554688
[step: 231] loss:

[step: 328] loss: 0.007103754207491875
[step: 329] loss: 113.70221710205078
[step: 329] loss: 0.007102352567017078
[step: 330] loss: 116.52227783203125
[step: 330] loss: 0.007101037073880434
[step: 331] loss: 123.85237121582031
[step: 331] loss: 0.007099774666130543
[step: 332] loss: 126.11654663085938
[step: 332] loss: 0.0070985471829771996
[step: 333] loss: 130.03289794921875
[step: 333] loss: 0.007097398862242699
[step: 334] loss: 126.9283676147461
[step: 334] loss: 0.007096346002072096
[step: 335] loss: 125.39501190185547
[step: 335] loss: 0.007095457520335913
[step: 336] loss: 115.04151916503906
[step: 336] loss: 0.007094911765307188
[step: 337] loss: 113.41513061523438
[step: 337] loss: 0.007094970438629389
[step: 338] loss: 121.15536499023438
[step: 338] loss: 0.007096193265169859
[step: 339] loss: 124.38172912597656
[step: 339] loss: 0.007099704816937447
[step: 340] loss: 123.47400665283203
[step: 340] loss: 0.007107636425644159
[step: 341] loss: 112.25129699707031
[step: 341] 

[step: 438] loss: 93.02013397216797
[step: 438] loss: 0.007094082422554493
[step: 439] loss: 94.24794006347656
[step: 439] loss: 0.007045503705739975
[step: 440] loss: 94.56790924072266
[step: 440] loss: 0.006990831345319748
[step: 441] loss: 92.88137817382812
[step: 441] loss: 0.007050210144370794
[step: 442] loss: 91.49797058105469
[step: 442] loss: 0.007044696249067783
[step: 443] loss: 90.78596496582031
[step: 443] loss: 0.006987645290791988
[step: 444] loss: 91.05116271972656
[step: 444] loss: 0.007014842238277197
[step: 445] loss: 91.75627136230469
[step: 445] loss: 0.007035394199192524
[step: 446] loss: 92.09913635253906
[step: 446] loss: 0.006992433685809374
[step: 447] loss: 92.24153137207031
[step: 447] loss: 0.006990852765738964
[step: 448] loss: 91.44105529785156
[step: 448] loss: 0.007019272074103355
[step: 449] loss: 90.57563781738281
[step: 449] loss: 0.006998987402766943
[step: 450] loss: 89.72496032714844
[step: 450] loss: 0.00697923032566905
[step: 451] loss: 89.32383

...

[step: 3473] loss: 0.006039425265043974
[step: 3474] loss: 23.2877197265625
[step: 3474] loss: 0.006047660019248724
[step: 3475] loss: 24.377267837524414
[step: 3475] loss: 0.006055389530956745
[step: 3476] loss: 25.78015899658203
[step: 3476] loss: 0.006063910201191902
[step: 3477] loss: 27.374126434326172
[step: 3477] loss: 0.006069638300687075
[step: 3478] loss: 28.67881202697754
[step: 3478] loss: 0.0060734315775334835
[step: 3479] loss: 27.074993133544922
[step: 3479] loss: 0.006070797331631184
[step: 3480] loss: 25.011425018310547
[step: 3480] loss: 0.006063200533390045
[step: 3481] loss: 23.066238403320312
[step: 3481] loss: 0.006049238611012697
[step: 3482] loss: 22.898582458496094
[step: 3482] loss: 0.006033336278051138
[step: 3483] loss: 22.368450164794922
[step: 3483] loss: 0.006018560379743576
[step: 3484] loss: 21.367801666259766
[step: 3484] loss: 0.0060089752078056335
[step: 3485] loss: 20.417387008666992
[step: 3485] loss: 0.006005610805004835
[step: 3486] loss: 20.8427

[step: 3582] loss: 316.218017578125
[step: 3582] loss: 0.006137957330793142
[step: 3583] loss: 376.513671875
[step: 3583] loss: 0.006159601267427206
[step: 3584] loss: 320.49090576171875
[step: 3584] loss: 0.006167816463857889
[step: 3585] loss: 299.3219909667969
[step: 3585] loss: 0.006139995064586401
[step: 3586] loss: 266.8534851074219
[step: 3586] loss: 0.006090280134230852
[step: 3587] loss: 232.86595153808594
[step: 3587] loss: 0.006033869460225105
[step: 3588] loss: 283.3967590332031
[step: 3588] loss: 0.006000205874443054
[step: 3589] loss: 190.48361206054688
[step: 3589] loss: 0.006000147201120853
[step: 3590] loss: 245.29583740234375
[step: 3590] loss: 0.0060234712436795235
[step: 3591] loss: 163.70266723632812
[step: 3591] loss: 0.00604935921728611
[step: 3592] loss: 196.069580078125
[step: 3592] loss: 0.006059733685106039
[step: 3593] loss: 175.913330078125
[step: 3593] loss: 0.0060506900772452354
[step: 3594] loss: 173.31439208984375
[step: 3594] loss: 0.006026891525834799

[step: 3690] loss: 0.006031686905771494
[step: 3691] loss: 45.75798416137695
[step: 3691] loss: 0.0060292514972388744
[step: 3692] loss: 48.56324005126953
[step: 3692] loss: 0.0060158371925354
[step: 3693] loss: 53.041778564453125
[step: 3693] loss: 0.005999201908707619
[step: 3694] loss: 63.78356170654297
[step: 3694] loss: 0.00598460016772151
[step: 3695] loss: 67.06359100341797
[step: 3695] loss: 0.005976792890578508
[step: 3696] loss: 65.98654174804688
[step: 3696] loss: 0.005976658780127764
[step: 3697] loss: 48.60273742675781
[step: 3697] loss: 0.00598220806568861
[step: 3698] loss: 44.50693130493164
[step: 3698] loss: 0.005989991594105959
[step: 3699] loss: 54.633384704589844
[step: 3699] loss: 0.005996360443532467
[step: 3700] loss: 51.50383377075195
[step: 3700] loss: 0.005999854765832424
[step: 3701] loss: 42.86558532714844
[step: 3701] loss: 0.0059990789741277695
[step: 3702] loss: 46.94940948486328
[step: 3702] loss: 0.0059959255158901215
[step: 3703] loss: 49.9293670654296

[step: 3799] loss: 30.399864196777344
[step: 3799] loss: 0.005970959551632404
[step: 3800] loss: 35.66645431518555
[step: 3800] loss: 0.005973817780613899
[step: 3801] loss: 33.638160705566406
[step: 3801] loss: 0.005977464374154806
[step: 3802] loss: 30.229406356811523
[step: 3802] loss: 0.0059822238981723785
[step: 3803] loss: 34.489532470703125
[step: 3803] loss: 0.005989376921206713
[step: 3804] loss: 31.264808654785156
[step: 3804] loss: 0.0059989988803863525
[step: 3805] loss: 30.363914489746094
[step: 3805] loss: 0.006013268139213324
[step: 3806] loss: 33.1983528137207
[step: 3806] loss: 0.006029970478266478
[step: 3807] loss: 29.930526733398438
[step: 3807] loss: 0.0060529643669724464
[step: 3808] loss: 30.388858795166016
[step: 3808] loss: 0.006074736826121807
[step: 3809] loss: 31.66248321533203
[step: 3809] loss: 0.006098330952227116
[step: 3810] loss: 29.160602569580078
[step: 3810] loss: 0.006106930319219828
[step: 3811] loss: 29.574745178222656
[step: 3811] loss: 0.006101

[step: 3907] loss: 0.005955965723842382
[step: 3908] loss: 25.055988311767578
[step: 3908] loss: 0.005950275342911482
[step: 3909] loss: 25.014320373535156
[step: 3909] loss: 0.005947896745055914
[step: 3910] loss: 24.73632049560547
[step: 3910] loss: 0.005949018523097038
[step: 3911] loss: 24.35363006591797
[step: 3911] loss: 0.005952092818915844
[step: 3912] loss: 24.346973419189453
[step: 3912] loss: 0.005955090746283531
[step: 3913] loss: 24.70697784423828
[step: 3913] loss: 0.005956642795354128
[step: 3914] loss: 24.852218627929688
[step: 3914] loss: 0.005957053042948246
[step: 3915] loss: 24.607446670532227
[step: 3915] loss: 0.005956733133643866
[step: 3916] loss: 24.263532638549805
[step: 3916] loss: 0.005956451408565044
[step: 3917] loss: 24.296993255615234
[step: 3917] loss: 0.005956139415502548
[step: 3918] loss: 24.52581214904785
[step: 3918] loss: 0.005955767352133989
[step: 3919] loss: 24.479875564575195
[step: 3919] loss: 0.005954854190349579
[step: 3920] loss: 24.432937

In [2]:
np.min(loss_for_graph)

19.296037673950195